## ***Optimization Techniques***

*   Gradient Descent algorithm

*   Stochastic Gradient Descent algorithm
*   BFGS algorithm

*   LBFGS algorithm



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.io
import random

In [ ]:
# Load the data from CSV files
x_tr = pd.read_csv('/kaggle/input/data-set/Final _dataset/X_train.csv', header=None).values
print("x_tr", x_tr.shape)

y_tr = pd.read_csv('/kaggle/input/data-set/Final _dataset/Y_train.csv', header=None).values.flatten()
y_tr = y_tr.reshape(y_tr.shape[0],1)
print("y_tr", y_tr.shape)

x_test = pd.read_csv('/kaggle/input/data-set/Final _dataset/X_test.csv', header=None).values
print("x_test", x_test.shape)

y_test = pd.read_csv('/kaggle/input/data-set/Final _dataset/Y_test.csv', header=None).values.flatten()
y_test = y_test.reshape(y_test.shape[0],1)
print("y_test", y_test.shape)

x_tr (10000, 784)
y_tr (10000, 1)
x_test (1000, 784)
y_test (1000, 1)


In [ ]:
def sigma_sq(Z):
    sigma_sq = 0
    for i in range(len(Z)):
        for j in range(len(Z)):
            sigma_sq += (np.linalg.norm(Z[i] - Z[j]) ** 2)
    sigma_sq = sigma_sq / (len(Z) ** 2)
    return sigma_sq

In [ ]:
# sigma square for x_train data set

sigma_sq(x_tr)

# Saved the results as train_sigma_sq to avoid re-running

103.46796422725427

In [ ]:
# sigma square for x_test data set

sigma_sq(x_test)

# Saved the results as test_sigma_sq to avoid re-running

102.81865457345208

In [ ]:
# Saved sigma_square value for train and test data to avoid repetative running

train_sigma_sq = math.sqrt(103.468)
test_sigma_sq = math.sqrt(102.818)

In [ ]:
# Define RBF kernel
def rbf_kernel(x1, x2, sigma_sq):
    Rbf_K = np.exp(-np.linalg.norm(x1 - x2) ** 2 / (2 * sigma_sq ** 2))

    return Rbf_K

In [ ]:
def training_k(x_tr, sigma_sq):
    S = len(x_tr)
    tr_k = np.zeros((S, S))

    for i in range(S):
        for j in range(i, S):
            tr_k[i, j] = rbf_kernel(x_tr[i], x_tr[j], sigma_sq)
            tr_k[j, i] = tr_k[i, j]

    return tr_k

In [ ]:
def testing_k(test_x, train_x, sigma_sq):
    test_k = np.zeros((len(test_x), len(train_x)))

    for i in range(len(test_x)):
        for j in range(len(train_x)):

            test_k[i, j] = rbf_kernel(test_x[i], train_x[j], sigma_sq)

    return test_k

In [ ]:
# kernel for x_train data set

Train_k = training_k(x_tr, train_sigma_sq)
np.save('Train_k.npy', Train_k)

In [ ]:
# kernel for x_test data set

Test_k = testing_k(x_test, x_tr, test_sigma_sq)
np.save('Test_k.npy',Test_k)

In [ ]:
# Load the presaved kernel test and train files

Train_k = np.load('/kaggle/input/data-set/Final _dataset/k_train.npy')
Test_k = np.load('/kaggle/input/data-set/Final _dataset/k_test.npy')

In [ ]:
# sigmoid func
def sigmoid(z):
    sig = 1 / (1 + np.exp(-np.clip(z, -100, 100)))
    return sig

In [ ]:
#  loss func
def loss(w, y, kernel, lamb):
    z = np.multiply(y, np.dot(kernel, w))
    Loss = -np.sum(np.log(sigmoid(z))) + lamb * np.dot(w.T, w)
    return Loss

In [ ]:
# Gradient
def gradient(w, y, kernel, lamb):
    z = np.multiply(y, np.dot(kernel, w))
    grad = -np.dot(kernel.T, np.multiply(y, (1 - sigmoid(z)))) + 2 * lamb * w
    return grad

* **GD_Algorithm**

In [ ]:
def GD_train(kernel, y, sigma_sq, lamb, lr):

    loss_total = []
    accuracy_tr = []
    accuracy_test = []
    I, J = kernel.shape
    accuracy = 0
    w_out = np.zeros((I,1))

    w = np.zeros((I,1))
    y = y.reshape(I,1)
    threshold = 1e-5
    iteration = 1200

    for epoch in range(iteration):
        prev_w = np.copy(w)
        grad = gradient(w, y, kernel, lamb)
        w -= lr * grad

        tr_loss = loss(w, y, kernel, lamb)
        tr_out = predict(kernel, w)
        tr_a = np.mean(tr_out == y)
        test_out = predict(Test_k, w)
        test_a = np.mean(test_out == y_test.reshape(1000,1))

        if test_a > accuracy:
            accuracy = test_a
            w_out = np.copy(w)
        loss_total.append(tr_loss)
        accuracy_tr.append(tr_a)
        accuracy_test.append(test_a)

        if (epoch) % 50 == 0:
            print(f"Epoch {epoch}: Train Loss = {tr_loss[0][0]:.3f} \t Train Accuracy = {tr_a*100:.2f} %")

        if np.linalg.norm(grad) < threshold:
            print(f"Converged at Epoch {epoch+1}")
            break

    return w_out

In [ ]:
# Testing
def predict(kernel, w):

    y_out = sigmoid(np.dot(kernel, w))
    y_out[y_out > 0.5] = 1
    y_out[y_out <= 0.5] = -1
    return y_out

In [ ]:
# Train
w = GD_train(Train_k, y_tr, train_sigma_sq, lamb = 0.005, lr = 0.0005)
print()

# Testing the accuracy
y_out = predict(Test_k,w)
Test_accuracy = np.mean(y_out == y_test)
print()
print(f"GD_test_accuracy = {Test_accuracy * 100 : .2f}%")

Epoch 0: Train Loss = 500000.204 	 Train Accuracy = 50.00 %
Epoch 50: Train Loss = 500499.145 	 Train Accuracy = 50.00 %
Epoch 100: Train Loss = 501621.540 	 Train Accuracy = 50.00 %
Epoch 150: Train Loss = 478489.693 	 Train Accuracy = 52.23 %
Epoch 200: Train Loss = 144195.640 	 Train Accuracy = 85.47 %
Epoch 250: Train Loss = 137552.802 	 Train Accuracy = 86.20 %
Epoch 300: Train Loss = 130265.237 	 Train Accuracy = 86.81 %
Epoch 350: Train Loss = 125582.904 	 Train Accuracy = 87.31 %
Epoch 400: Train Loss = 121219.833 	 Train Accuracy = 87.67 %
Epoch 450: Train Loss = 116857.064 	 Train Accuracy = 88.12 %
Epoch 500: Train Loss = 113251.137 	 Train Accuracy = 88.20 %
Epoch 550: Train Loss = 483141.529 	 Train Accuracy = 51.87 %
Epoch 600: Train Loss = 133545.672 	 Train Accuracy = 86.68 %
Epoch 650: Train Loss = 120614.824 	 Train Accuracy = 87.99 %
Epoch 700: Train Loss = 118218.393 	 Train Accuracy = 88.23 %
Epoch 750: Train Loss = 115129.797 	 Train Accuracy = 88.50 %
Epoch 800: 

* **SGD_Algorithm**

In [ ]:
def SGD_train(Train_k, y_tr, Test_k, y_test, lamb, lr, b_s):

    loss_tr = []
    loss_test = []
    accuracy_tr = []
    accuracy_test = []

    I = Train_k.shape[0]
    J = Test_k.shape[0]
    accuracy = 0
    w_out = np.zeros((I,1))

    w = np.zeros((I,1))
    y_tr = y_tr.reshape(I,1)
    y_test = y_test.reshape(J,1)
    threshold = 1e-5
    iterations = 1200

    for epoch in range(iterations):
        prev_w = np.copy(w)
        indices = np.random.permutation(I)
        num_batches = int(np.ceil(I / b_s))

        for i in range(num_batches):
            batch_indices = indices[i*b_s:(i+1)*b_s]
            batch_K_Train = Train_k[batch_indices,:]
            batch_y_tr = y_tr[batch_indices,:]
            grad = gradient(w, batch_y_tr, batch_K_Train, lamb)
            w -= lr * grad

        tr_loss = loss(w, y_tr, Train_k, lamb)
        test_loss = loss(w, y_test, Test_k, lamb)
        tr_out = predict(Train_k, w)
        test_out = predict(Test_k, w)
        tr_a = np.mean(tr_out == y_tr)
        test_a = np.mean(test_out == y_test)


        if test_a > accuracy:
            accuracy = test_a
            w_out = np.copy(w)
        loss_tr.append(tr_loss)
        loss_test.append(test_loss)
        accuracy_tr.append(tr_a)
        accuracy_test.append(test_a)

        if (epoch) % 50 == 0:
            print(f"Epoch {epoch}: Train Loss = {tr_loss[0][0]:.3f} \t Train Accuracy = {tr_a*100:.2f} %")

        if (w - prev_w).any() < threshold:
            print(f"Converge at {epoch+1}")
            break

    return w_out

In [ ]:
w = SGD_train(Train_k, y_tr, Test_k, y_test, lamb = 0.005, lr = 0.0005, b_s=1)

# Testing accuracy
print()
y_out = predict(Test_k,w)
accuracy = np.mean(y_out == y_test)
print(f"Accuracy = {accuracy * 100:.2f}%")

Epoch 0: Train Loss = 3133.938 	 Train Accuracy = 87.24 %
Epoch 50: Train Loss = 2389.262 	 Train Accuracy = 90.98 %
Epoch 100: Train Loss = 2683.745 	 Train Accuracy = 89.62 %
Epoch 150: Train Loss = 3702.762 	 Train Accuracy = 84.02 %
Epoch 200: Train Loss = 2583.018 	 Train Accuracy = 90.36 %
Epoch 250: Train Loss = 2376.595 	 Train Accuracy = 91.25 %
Epoch 300: Train Loss = 2461.731 	 Train Accuracy = 90.86 %
Epoch 350: Train Loss = 2407.520 	 Train Accuracy = 91.14 %
Epoch 400: Train Loss = 2410.969 	 Train Accuracy = 90.81 %
Epoch 450: Train Loss = 2570.672 	 Train Accuracy = 90.43 %
Epoch 500: Train Loss = 2521.798 	 Train Accuracy = 90.33 %
Epoch 550: Train Loss = 2514.721 	 Train Accuracy = 90.33 %
Epoch 600: Train Loss = 2505.142 	 Train Accuracy = 90.61 %
Epoch 650: Train Loss = 2417.261 	 Train Accuracy = 90.81 %
Epoch 700: Train Loss = 2388.084 	 Train Accuracy = 90.86 %
Epoch 750: Train Loss = 2527.004 	 Train Accuracy = 90.54 %
Epoch 800: Train Loss = 3026.159 	 Train Ac

In [ ]:
w = SGD_train(Train_k, y_tr, Test_k, y_test, lamb = 0.005, lr = 0.0005, b_s = 100)

# Testing accuracy
print()
y_out = predict(Test_k,w)
accuracy = np.mean(y_out == y_test)
print(f"Accuracy = {accuracy * 100:.2f}%")

Epoch 0: Train Loss = 187811.998 	 Train Accuracy = 50.00 %
Epoch 50: Train Loss = 52826.973 	 Train Accuracy = 68.45 %
Epoch 100: Train Loss = 14752.435 	 Train Accuracy = 87.48 %
Epoch 150: Train Loss = 9864.406 	 Train Accuracy = 90.16 %
Epoch 200: Train Loss = 8803.216 	 Train Accuracy = 91.65 %
Epoch 250: Train Loss = 13060.206 	 Train Accuracy = 88.77 %
Epoch 300: Train Loss = 96785.761 	 Train Accuracy = 56.76 %
Epoch 350: Train Loss = 9019.100 	 Train Accuracy = 91.59 %
Epoch 400: Train Loss = 10683.480 	 Train Accuracy = 90.03 %
Epoch 450: Train Loss = 77567.776 	 Train Accuracy = 62.64 %
Epoch 500: Train Loss = 7599.842 	 Train Accuracy = 92.44 %
Epoch 550: Train Loss = 9932.924 	 Train Accuracy = 90.15 %
Epoch 600: Train Loss = 7089.271 	 Train Accuracy = 92.60 %
Epoch 650: Train Loss = 8699.906 	 Train Accuracy = 91.26 %
Epoch 700: Train Loss = 7645.266 	 Train Accuracy = 92.54 %
Epoch 750: Train Loss = 7178.130 	 Train Accuracy = 92.73 %
Epoch 800: Train Loss = 7904.301 	 

* **BFGS_Algorithm**

In [ ]:
# Random data saving
np.random.seed(14)
random.seed(14)
Inced_two = np.where(y_tr == 1)[0]
Index_two = np.where(y_tr == -1)[0]
np.random.shuffle(Inced_two)
np.random.shuffle(Index_two)
Inced_two = Inced_two[:2000]
Index_two = Index_two[:2000]

index_s = np.concatenate([Inced_two, Index_two])
bfgs_x_tr = x_tr[index_s]
bfgs_y_tr = y_tr[index_s]

In [ ]:
# BFGS train kernel

bfgs_k_train = training_k(bfgs_x_tr, train_sigma_sq)
np.save('bfgs_k_train.npy', bfgs_k_train)

# Saved as document to avoid re-running

In [ ]:
# BFGS test kernel

bfgs_k_test = testing_k(x_test, bfgs_x_tr, test_sigma_sq)
np.save('bfgs_k_test.npy', bfgs_k_test)

# Saved as document to avoid re-running

In [ ]:
# Load the saved BFGS kernel data for training and testing

bfgs_k_train = np.load('/kaggle/input/bgfsdata/bfgs_k_train.npy')
bfgs_k_test = np.load('/kaggle/input/bgfsdata/bfgs_k_test.npy')

In [ ]:
def find_l(w, ker_p, y_tr, Train_k, lamb, in_alp = 1, c = 0.1, P = 0.5):

    Alp = in_alp
    iteration = 100
    ind = 0
    ker_f = loss(w, y_tr, Train_k, lamb)
    ker_g = np.dot(gradient(w, y_tr, Train_k, lamb).T, ker_p)

    for i in range(iteration):
        w_new = w + Alp * ker_p
        ker_f_new = loss(w_new, y_tr, Train_k, lamb)

        if ker_f_new <= ker_f + c * Alp * ker_g:
            break
        Alp *= P
        ind += 1
    return Alp

In [ ]:
def BFGS_train(Train_k, y_tr, Test_k, y_test, lamb):

    loss_tr = []
    loss_test = []
    accuracy_tr = []
    accuracy_test = []

    p = Train_k.shape[0]
    q = Test_k.shape[0]
    iteration = 200
    threshold = 1e-5
    accuracy = 0
    w_out = np.zeros((p,1))


    # Weight and Hessian initialization
    w = np.zeros((p,1))
    y_tr = y_tr.reshape(p,1)
    y_test = y_test.reshape(q,1)
    Ki = np.eye(p)
    Kh = Ki

    for epoch in range(iteration):
        gd = gradient(w, y_tr, Train_k, lamb)
        ker_p = -np.dot(Kh, gd)
        Alp = find_l(w, ker_p, y_tr, Train_k, lamb)
        new_w = w + Alp * ker_p
        sk = new_w - w
        yk = gradient(new_w, y_tr, Train_k, lamb) - gd
        P = 1 / np.dot(yk.T, sk)
        Ka1 = Ki - P * np.outer(sk, yk)
        Ka2 = Ki - P * np.outer(yk, sk)
        Ka3 = P * np.outer(sk, sk)
        Kh = np.dot(Ka1, np.dot(Kh, Ka2)) + Ka3

        w = new_w
        tr_loss = loss(w, y_tr, Train_k, lamb)
        train_out = predict(Train_k, w)
        tr_a = np.mean(train_out == y_tr)
        test_loss = loss(w, y_test, Test_k, lamb)
        test_out = predict(Test_k, w)
        test_a = np.mean(test_out == y_test)

        if test_a > accuracy:
            accuracy = test_a
            w_out = np.copy(w)
        loss_tr.append(tr_loss)
        loss_test.append(test_loss)
        accuracy_tr.append(tr_a)
        accuracy_test.append(test_a)

        if (epoch) % 20 == 0:
            print(f"Epoch {epoch}: Train Loss = {tr_loss[0][0]:.3f} \t Train Accuracy = {tr_a*100:.2f} %")

        if sk.any() < threshold:
            print(f"Converged at Epoch {epoch+1}")
            break

    return w_out

In [ ]:
w = BFGS_train(bfgs_k_train, bfgs_y_tr, bfgs_k_test, y_test, lamb = 0.005)


Epoch 0: Train Loss = 2755.877 	 Train Accuracy = 50.00 %
Epoch 20: Train Loss = 585.327 	 Train Accuracy = 95.33 %
Epoch 40: Train Loss = 411.110 	 Train Accuracy = 97.85 %
Epoch 60: Train Loss = 396.547 	 Train Accuracy = 98.28 %
Epoch 80: Train Loss = 396.425 	 Train Accuracy = 98.30 %
Epoch 100: Train Loss = 396.424 	 Train Accuracy = 98.30 %
Epoch 120: Train Loss = 396.424 	 Train Accuracy = 98.30 %
Epoch 140: Train Loss = 396.424 	 Train Accuracy = 98.30 %
Epoch 160: Train Loss = 396.424 	 Train Accuracy = 98.30 %
Epoch 180: Train Loss = 396.424 	 Train Accuracy = 98.30 %


In [ ]:
print("BFGS_Test_Accuracy = " + str(accuracy*100) + "%")

BFGS_Test_Accuracy = 94.3%


* **LBFGS_Algorithm**

In [ ]:
def lbfgs_direction(gd, His_s, His_y, His_p, hist_s):
    q = gd.copy()
    alpha_list = [0] * hist_s

    for i in range(len(His_s)-1, -1, -1):
        rho = His_p[i]
        alpha_list[i] = rho * np.dot(His_s[i].T, q)
        q = q - alpha_list[i] * His_y[i]
    Hk_0 = np.dot(His_y[-1].T, His_s[-1]) / np.dot(His_y[-1].T, His_y[-1])
    r = Hk_0 * q

    for i in range(len(His_s)):
        rho = His_p[i]
        beta = rho * np.dot(His_y[i].T, r)
        r = r + His_s[i] * (alpha_list[i] - beta)
    pk = -r
    return pk

In [ ]:
def train_LBFGS(Train_k, y_tr, Test_k, y_test, lamb, hist_s):

    iteration = 200
    tolerance = 1e-5
    p = Train_k.shape[0]
    q = Test_k.shape[0]

    loss_tr = []
    loss_test = []
    accuracy_tr = []
    accuracy_test = []
    accuracy = 0
    w_out = np.zeros((p,1))

    w = np.zeros((p,1))
    y_tr = y_tr.reshape(p,1)
    y_test = y_test.reshape(q,1)
    His_s = []
    His_y = []
    His_p = []

    for epoch in range(iteration):
        gd = gradient(w, y_tr, Train_k, lamb)
        if epoch == 0:
            pk = -gd
        else:
            pk = lbfgs_direction(gd, His_s, His_y, His_p, hist_s)
        alpha = find_l(w, pk, y_tr, Train_k, lamb)
        new_w = w + alpha * pk
        sk = new_w - w
        yk = gradient(new_w, y_tr, Train_k, lamb) - gd
        rho = 1 / np.dot(yk.T, sk)

        if len(His_s) == hist_s:
            His_s.pop(0)
            His_y.pop(0)
            His_p.pop(0)
        His_s.append(sk)
        His_y.append(yk)
        His_p.append(rho)
        w = new_w
        tr_loss = loss(w, y_tr, Train_k, lamb)
        train_out = predict(Train_k, w)
        tr_a = np.mean(train_out == y_tr)
        test_loss = loss(w, y_test, Test_k, lamb)
        test_out = predict(Test_k, w)
        test_a = np.mean(test_out == y_test)

        if test_a > accuracy:
            accuracy = test_a
            w_out = np.copy(w)
        loss_tr.append(tr_loss)
        loss_test.append(test_loss)
        accuracy_tr.append(tr_a)
        accuracy_test.append(test_a)

        if (epoch) % 20 == 0:
            print(f"Epoch {epoch}: Train Loss = {tr_loss[0][0]:.3f} \t Train Accuracy = {tr_a*100:.2f} %")
        # Check for convergence
        if np.linalg.norm(sk) < tolerance:
            print(f"Converged at Epoch {epoch+1}")
            break

    return w_out

In [ ]:
w = train_LBFGS(bfgs_k_train, bfgs_y_tr, bfgs_k_test, y_test, lamb = 0.005, hist_s = 10)
print()
print("LBFGS_Test_Accuracy = " + str(accuracy*100) + "%")

Epoch 0: Train Loss = 2755.877 	 Train Accuracy = 50.00 %
Epoch 20: Train Loss = 1019.697 	 Train Accuracy = 90.53 %
Epoch 40: Train Loss = 931.048 	 Train Accuracy = 90.85 %
Epoch 60: Train Loss = 841.165 	 Train Accuracy = 92.35 %
Epoch 80: Train Loss = 803.388 	 Train Accuracy = 92.67 %
Epoch 100: Train Loss = 773.399 	 Train Accuracy = 92.70 %
Epoch 120: Train Loss = 753.387 	 Train Accuracy = 93.20 %
Epoch 140: Train Loss = 740.630 	 Train Accuracy = 93.42 %
Epoch 160: Train Loss = 721.581 	 Train Accuracy = 93.80 %
Epoch 180: Train Loss = 701.562 	 Train Accuracy = 94.00 %

LBFGS_Test_Accuracy = 94.3%


.

The optimization algorithms are used to minimize the value of the cost function with respect to time. For different methods the reduction in cost with time vary based on optimization methods used.

* From the above solution we can deduce that LBFGS is suitable for large-scale optimization problems as it is memory-efficient version of BFGS and also converges faster than GD and SGD and also converges smoothly.But, It requires Requires more memory compared to GD and SGD.

*   The cost function for BFGS decreases smoothly over time.  BFGS can converge faster than GD and SGD, especially when the cost function is smooth and convex. But, requires more memory than GD and SGD because of Hessian approximation.

*   Gradient Descent on the other hand is a simple and widely used optimization algorithm for minimizing the cost function, and the Cost function decreases smoothly over time. But, has a tendency to converge to a local minimum for non-convex function, and can be slow when the dataset is large.

*   SGD is a variation of GD that randomly samples a subset of the data to compute the gradient which makes SGD faster than GD, especially when the dataset is large. However, because it uses only a subset of the data, the update directionis noisy and lead to fluctuations in the cost function.But this niose also helps SGD to escape local minima and converge to a better global minimum compared to GD.



.